# python处理CSV/JSON/XML数据

csv、json和xml是三种主流的数据格式。

## csv数据

存储数据最常用的格式（kaggle比赛的大部分数据格式）。可以使用python自带的csv库读取和写入：

### csv数据的读取

In [49]:
import csv

filename  = "datasets/mnist_test.csv"

filed = []
rows = []

# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # extracting filed names in the first row
    filed = next(csvreader)

    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)

# printing out the first row
# for row in rows[:1]:
#     print(row)


1. 教程中有错误：在python3中，csvreader的下一个读取方式不再是"csvreader.next()"而是next(csvreader)

### csv数据的写入

In [50]:
# field names
fileds = ['Name', 'Goals', 'Assists', 'Shots']

# rows of data in the csv file
rows = [['Emily', '12', '18', '112'],
        ['Katie', '8', '24', '96'],
        ['John', '16', '9', '101'],
        ['Mike', '3', '14', '82']]

# writing to csv file
with open("datasets/soccer.csv", 'w+', newline='') as csvfile:  # add "new=''" to avoid blank row
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fileds
    csvwriter.writerow(fileds)

    # writing the data rows
    csvwriter.writerows(rows)

# 读取写入的csv数据
filed = []
rows = []

with open("datasets/soccer.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    filed = next(csvfile)
    for row in csvreader:
        rows.append(row)
print(filed)
for row in rows:
    print(row)



Name,Goals,Assists,Shots

['Emily', '12', '18', '112']
['Katie', '8', '24', '96']
['John', '16', '9', '101']
['Mike', '3', '14', '82']


1. csv的方法"writerow"和"writerows"的用法不同，一维和二维  
2. 打开文件时加上“newline=''”才不会有空行

### 用pandas将csv格式的数据转化

In [18]:
import pandas as pd

# header=0: 以第0行的数据为列名

dic_from_csv = pd.read_csv('datasets/soccer.csv', header=0).to_dict('records')
print(dic_from_csv)

[{'Name': 'Emily', 'Goals': 12, 'Assists': 18, 'Shots': 112}, {'Name': 'Katie', 'Goals': 8, 'Assists': 24, 'Shots': 96}, {'Name': 'John', 'Goals': 16, 'Assists': 9, 'Shots': 101}, {'Name': 'Mike', 'Goals': 3, 'Assists': 14, 'Shots': 82}]


1. 加上“orient='records'”，否则字典构造会自动加上列索引，影响xml文件的转换

### csv数据转化为其他数据

通过上一步，csv转化为了dict，再通过dict写json和xml文件

In [26]:
# csv to json
import json

with open("datasets/soccer.json", 'w+') as json_file:
    json.dump(dic_from_csv, json_file, indent=4)

# csv to xml
from dicttoxml import dicttoxml

# decode the dict and convert it to str
# then write a xml file
xml_data = dicttoxml(dic_from_csv).decode()
with open("datasets/soccer.xml", "w+") as xml_file:
    xml_file.write(xml_data)

## json数据

json数据简洁易读，保持了字典结构。python中同样有内置json模块

### json数据的读取

In [21]:
# read the data from file
# we now have a python dictionary
with open('datasets/soccer.json') as f:
    data_listofdict = json.load(f)

# we can do the same thing with pandas
# pandas read the file, turn it to df(dataframe)
data_df = pd.read_json('datasets/soccer.json', orient='records')
print(data_df)

    Name  Goals  Assists  Shots
0  Emily     12       18    112
1  Katie      8       24     96
2   John     16        9    101
3   Mike      3       14     82


### json数据的写入

In [47]:
# we can write a dictionary to json like so
# use 'indent'  and 'sort_keys' to make the json
# file look nice
with open('datasets/new_soccer.json', 'w+') as json_file:
    json.dump(data_listofdict, json_file, indent=4)

# and again the same things with pandas
exports = data_df.to_json('datasets/new_soccer.json', orient='records')

## XML数据

xml数据和前两者不同。  
disadv:  
1.占用空间多  
2.传输和储存需要更大的带宽、更多的空间和运行时间  
adv:   
1.可以使用namespace构建、共享标准结构，更好继承

导入xml数据需要用到xml内置模块和子模ElementTree。xmltodict库可以将ElementTree转化为字典，从而转换为csv、xml、df。

### xml数据的读取

In [46]:
import xml.etree.ElementTree as ET
import xmltodict

tree = ET.parse('datasets/soccer.xml')
xml_data = tree.getroot()

xmlstr = ET.tostring(xml_data, encoding='utf8', method='xml')

data_dict = dict(xmltodict.parse(xmlstr))

print(data_dict)

{'root': OrderedDict([('item', [OrderedDict([('@type', 'dict'), ('Name', OrderedDict([('@type', 'str'), ('#text', 'Emily')])), ('Goals', OrderedDict([('@type', 'int'), ('#text', '12')])), ('Assists', OrderedDict([('@type', 'int'), ('#text', '18')])), ('Shots', OrderedDict([('@type', 'int'), ('#text', '112')]))]), OrderedDict([('@type', 'dict'), ('Name', OrderedDict([('@type', 'str'), ('#text', 'Katie')])), ('Goals', OrderedDict([('@type', 'int'), ('#text', '8')])), ('Assists', OrderedDict([('@type', 'int'), ('#text', '24')])), ('Shots', OrderedDict([('@type', 'int'), ('#text', '96')]))]), OrderedDict([('@type', 'dict'), ('Name', OrderedDict([('@type', 'str'), ('#text', 'John')])), ('Goals', OrderedDict([('@type', 'int'), ('#text', '16')])), ('Assists', OrderedDict([('@type', 'int'), ('#text', '9')])), ('Shots', OrderedDict([('@type', 'int'), ('#text', '101')]))]), OrderedDict([('@type', 'dict'), ('Name', OrderedDict([('@type', 'str'), ('#text', 'Mike')])), ('Goals', OrderedDict([('@typ

### xml数据的写入

In [48]:
xml_data = dicttoxml(data_listofdict).decode()
with open("datasets/new_soccer.xml", "w+") as xml_file:
    xml_file.write(xml_data)